# openai functions

Reference: https://github.com/Azure-Samples/openai/blob/main/Basic_Samples/Functions/working_with_functions.ipynb

In [1]:
import openai
import json

with open("openai_api.json") as f:
    creds = json.load(f)

openai.api_version = creds["api_version"]
openai.api_base = creds["api_base"]
openai.api_type = creds["api_type"]
openai.api_key = creds["api_key"]

## Natural language query to Search filters

In [20]:
system_prompt = """You are an AI assistant for an online shoe store that helps users find appropriate shoes.
You convert their queries into appropriate filters for the search function.
"""

functions = [
    {
        "name": "search",
        "description": "This function calls Search API of the shoe store to find top matches for user's description.",
        "parameters": {
            "type": "object",
            "properties": {
                "shoe_types": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["running", "jogging", "trekking", "sports", "heels"]
                    },
                    "description": "The types of shoes to search for."
                },
                "sizes": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["uk4", "uk5", "uk6", "uk7", "uk8", "uk9"]
                    },
                    "description": "The sizes of shoe to search for. Small refers to uk4 and uk5. Medium refers to uk6 and uk7. Large refers to uk8 and uk9."
                },
                "brands": {
                     "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["reebok", "puma", "adidas", "clarks london"]
                    },
                    "description": "The brands of shoes to search for."
                },
                "colors": {
                     "type": "array",
                    "items": {
                        "type": "string",
                        "enum": ["red", "green", "blue"]
                    },
                    "description": "The colors of shoes to search for."
                },
            }
        },
        "result": {
            "type": "array",
            "items": {
                "type": "string"
            }
        }
    }
]

In [27]:
def get_search_filters(chat_history):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        engine="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            *chat_history,
        ],
        temperature=0,
        functions=functions,
    )
    response_message = response.choices[0].message
    assert "function_call" in response_message
    search_filters = json.loads(response_message.function_call.arguments)
    
    chat_history.append({
        "role": response_message["role"],
        "function_call": {
            "name": response_message["function_call"]["name"],
            "arguments": response_message["function_call"]["arguments"],
        },
        "content": None
    })
    chat_history.append({
        "role": "function",
        "name": response_message["function_call"]["name"],
        "content": "List of 10 shoes with that description",
    })
    return search_filters, chat_history

In [30]:
user_queries = [
    "blue running shoes for small feet",
    "show nike and puma only",
    "include red",
    "show sports shoes instead of running"
]

chat_history = []
for query in user_queries:
    chat_history.append({"role": "user", "content": query})
    search_filters, chat_history = get_search_filters(chat_history)
    print(f"Query: {query}")
    print(f"Search Filters: {search_filters}")
    print()

Query: blue running shoes for small feet
Search Filters: {'shoe_types': ['running'], 'sizes': ['uk4', 'uk5'], 'colors': ['blue']}

Query: show nike and puma only
Search Filters: {'shoe_types': ['running'], 'sizes': ['uk4', 'uk5'], 'colors': ['blue'], 'brands': ['nike', 'puma']}

Query: include red
Search Filters: {'shoe_types': ['running'], 'sizes': ['uk4', 'uk5'], 'colors': ['blue', 'red'], 'brands': ['nike', 'puma']}

Query: show sports shoes instead of running
Search Filters: {'shoe_types': ['sports'], 'sizes': ['uk4', 'uk5'], 'colors': ['blue', 'red'], 'brands': ['nike', 'puma']}



## Chatbot actions

Sample chatbot created using openai functions. Chatbot works for an ecommerce app and helps customers solve order issues. 
It answers queries like:

- Where is my order?
- My delivered order has defects. I would like an exchange.
- Cancel my order
- List delivered orders. show details for order#2

In [2]:
"""
You are a sophisticated AI assistant, a specialist in user intent detection and interpretation.
Your task is to perceive and respond to the user's needs, even when they're expressed in an indirect or direct manner.
You excel in recognizing subtle cues: for example, if a user states they are 'hungry', you should assume they are seeking nearby dining options such as a restaurant or a cafe.
If they indicate feeling 'tired', 'weary', or mention a long journey, interpret this as a request for accommodation options like hotels or guest houses.
However, remember to navigate the fine line of interpretation and assumption: if a user's intent is unclear or can be interpreted in multiple ways, do not hesitate to politely ask for additional clarification.
Make sure to tailor your responses to the user based on their preferences and past experiences which can be found here {customer_profile_str}
"""

"\nYou are a sophisticated AI assistant, a specialist in user intent detection and interpretation.\nYour task is to perceive and respond to the user's needs, even when they're expressed in an indirect or direct manner.\nYou excel in recognizing subtle cues: for example, if a user states they are 'hungry', you should assume they are seeking nearby dining options such as a restaurant or a cafe.\nIf they indicate feeling 'tired', 'weary', or mention a long journey, interpret this as a request for accommodation options like hotels or guest houses.\nHowever, remember to navigate the fine line of interpretation and assumption: if a user's intent is unclear or can be interpreted in multiple ways, do not hesitate to politely ask for additional clarification.\nMake sure to tailor your responses to the user based on their preferences and past experiences which can be found here {customer_profile_str}\n"

In [3]:
        {
            "name": "call_google_places_api",
            "description": "This function calls the Google Places API to find the top places of a specified type near a specific location. It can be used when a user expresses a need (e.g., feeling hungry or tired) or wants to find a certain type of place (e.g., restaurant or hotel).",
            "parameters": {
                "type": "object",
                "properties": {
                    "place_type": {
                        "type": "string",
                        "description": "The type of place to search for."
                    }
                }
            },
            "result": {
                "type": "array",
                "items": {
                    "type": "string"
                }
            }
        }

{'name': 'call_google_places_api',
 'description': 'This function calls the Google Places API to find the top places of a specified type near a specific location. It can be used when a user expresses a need (e.g., feeling hungry or tired) or wants to find a certain type of place (e.g., restaurant or hotel).',
 'parameters': {'type': 'object',
  'properties': {'place_type': {'type': 'string',
    'description': 'The type of place to search for.'}}},
 'result': {'type': 'array', 'items': {'type': 'string'}}}

In [4]:
functions = [
    {
        "name": "track_order",
        "description": "Track the progress of the order. Where's the order currently and ETA for delivery.",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The Order ID."
                }
            }
        },
        "result": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "Current location of the order."
                },
                "eta": {
                    "type": "string",
                    "description": "Estimated time of arrival."
                }
            }
        }
    }
]

In [6]:
order_id = "order_1234"
user_input = "where is my order?"

system_prompt = """You are an AI assistant helping customers with their queries about orders for an ecommerce app.

"""

response = openai.ChatCompletion.create(
    model="gpt-4",
    engine="gpt-4",
    messages=[
        {
            "role": "system",
            "content": f""
        },
        {"role": "user", "content": user_input}
    ],
    temperature=0,
    functions=functions,
)